## Track Validation

written by Isobel Mawby (i.mawby1@lancaster.ac.uk)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Imports
</div>

In [ ]:
import random
import uproot
import numpy as np
import math
import matplotlib.pyplot as plt
import awkward as ak

%matplotlib widget
from termcolor import colored, cprint

import Definitions
import ValidationFunc
import TrackValidationFunc

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Config
</div>

In [ ]:
SHOW_PLOTS = False

<div class="alert alert-block alert-info" style="font-size: 18px;">
    File
</div>

In [ ]:
file_name = "/Users/isobel/Desktop/DUNE/2026/PandoraValidation/files/ValidationOldHierarchy.root"

plot_dir = '/Users/isobel/Desktop/DUNE/2026/PandoraValidation/TrackValPlots/'

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Lets open the file...
</div>

In [ ]:
file = uproot.open(file_name)

In [ ]:
event_tree = file['EventTree']
pfp_tree = file['PFPTree']
track_tree = file['TrackTree']
hierarchy_tree = file['HierarchyTree']

event_branches = event_tree.arrays(['Run', 'Subrun', 'Event', 'MCInt_IsCC', 'MCNu_PDG'], library="ak")
pfp_branches = pfp_tree.arrays(['Run', 'Subrun', 'Event',
                                'MCP_TruePDG', 'MCP_HasMatch', 'MCP_NMCHits2D',
                                'BM_Completeness', 'BM_Purity', 'BM_IsTrack', 'BM_IsShower'], library="ak")
track_branches = track_tree.arrays(['MCP_HasMichel', 'MCP_HasTargetMichel', 'BM_IsMichelRecod', 'BM_MichelIndex', 'BM_MichelIsChild', 'BM_MichelIsShower'], library="ak")
hierarchy_branches = hierarchy_tree.arrays(['MC_HierarchyTier', 'MC_ParentIndex', 'BM_HierarchyTier', 'BM_ParentIndex'], library="ak")

In [ ]:
print(track_tree.keys())

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Summary
</div>

In [ ]:
int_masks = Definitions.GetIntMasks(event_branches, pfp_branches)
pdg_masks = Definitions.GetPDGMasks(pfp_branches)
tier_masks = Definitions.GetTierMasks(hierarchy_branches)

In [ ]:
#Definitions.PrintEventSummary(int_masks, hierarchy_branches, pfp_branches)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Plot Branch Variables
</div>

In [ ]:
# MCP branch variable plots
MCP_plotting_vars = [ValidationFunc.michel_n_mc_hits_2d_var, ValidationFunc.michel_completeness_var, ValidationFunc.michel_purity_var]
MCP_var_plots = [plt.subplots(ncols=len(Definitions.tiers), nrows=len(Definitions.ints), figsize=(14, 10)) for _ in MCP_plotting_vars]

# Track/shower classification plots
track_shower_vars = [ValidationFunc.michel_n_mc_hits_2d_var]
track_shower_var_plots = [plt.subplots(ncols=len(Definitions.tiers), nrows=len(Definitions.ints), figsize=(14, 10)) for _ in track_shower_vars]

# Efficiency table + plots
efficiency_file_name = f'MichelEfficiencyTables'
efficiency_vars = [ValidationFunc.michel_n_mc_hits_2d_var]
efficiency_plots = [plt.subplots(ncols=len(Definitions.tiers), nrows=len(Definitions.ints), figsize=(14, 10)) for _ in efficiency_vars]

# hierarchy table
hierarchy_file_name = f'MichelHierarchyTables'

with open(f"{plot_dir}{efficiency_file_name}.txt", "w") as f_efficiency, open(f"{plot_dir}{hierarchy_file_name}.txt", "w") as f_hierarchy:
    for int_type in Definitions.ints :
        int_mask = int_masks[int_type]

        ValidationFunc.PrintHierarchyTableHeader(int_type, f_hierarchy)
        ValidationFunc.PrintEfficiencyTableHeader(int_type, f_efficiency)
        
        for tier in Definitions.tiers :
            tier_mask = tier_masks[tier]        
            target_muon_mask = tier_mask & int_mask & pdg_masks[13] & (pfp_branches['MCP_HasMatch'] == 1) & (track_branches['MCP_HasTargetMichel'] == 1)
            target_muon_with_reco_michel_mask = target_muon_mask & (track_branches['BM_IsMichelRecod'] == 1)
            target_michel_indices = track_branches['BM_MichelIndex'][target_muon_mask]
            reco_michel_indices = track_branches['BM_MichelIndex'][target_muon_with_reco_michel_mask]

            # Hierarchy Metrics
            hierarchy_metrics = ValidationFunc.CalculateHierarchyMetrics(hierarchy_branches, reco_michel_indices)
            ValidationFunc.PrintHierarchyTableEntry(tier, hierarchy_metrics, f_hierarchy)         

            # Efficiency Metrics
            efficiency_metrics = ValidationFunc.CalculateEfficiencyMetrics(target_muon_mask, target_muon_with_reco_michel_mask)
            ValidationFunc.PrintEfficiencyTableEntry(tier, efficiency_metrics, f_efficiency)
            
            # Plot MCP_var distributions
            for i_var in range(len(MCP_plotting_vars)) :
                fig, axes = MCP_var_plots[i_var]
                ax = axes[int_type, tier]
                ValidationFunc.ConfigurePlot(fig, ax, int_type, tier, MCP_plotting_vars[i_var])
                ValidationFunc.plot_var(target_michel_indices, pfp_branches, MCP_plotting_vars[i_var], fig, ax, 'Michel')
                if not SHOW_PLOTS :
                    plt.close(fig)
    
            # Plot track/shower classifications
            for i_var in range(len(track_shower_vars)) :
                fig, axes = track_shower_var_plots[i_var]
                ax = axes[int_type, tier]
                ValidationFunc.ConfigurePlot(fig, ax, int_type, tier, track_shower_vars[i_var])
                ValidationFunc.TrackShowerAsAFunctionOf(reco_michel_indices, pfp_branches, ValidationFunc.michel_n_mc_hits_2d_var, fig, ax)
                if not SHOW_PLOTS :
                    plt.close(fig)

            # Plot efficiency
            for i_var in range(len(efficiency_vars)) :
                fig, axes = efficiency_plots[i_var]
                ax = axes[int_type, tier]
                ValidationFunc.ConfigurePlot(fig, ax, int_type, tier, efficiency_vars[i_var])
                TrackValidationFunc.plot_michel_efficiency(target_michel_indices, reco_michel_indices, pfp_branches, efficiency_vars[i_var], fig, ax)
                if not SHOW_PLOTS :
                    plt.close(fig)
                    
        ValidationFunc.PrintHierarchyTableFooter(f_hierarchy)
        ValidationFunc.PrintEfficiencyTableFooter(f_efficiency)

# Save MCP_var distributions
for i_var in range(len(MCP_plotting_vars)) :
    fig, _ = MCP_var_plots[i_var]
    file_name = f'Michel_{MCP_plotting_vars[i_var].tree_name}'
    fig.savefig(f'{plot_dir}{file_name}.pdf', bbox_inches='tight')

# Save track/shower classification plots
for i_var in range(len(track_shower_vars)) :
    fig, _ = track_shower_var_plots[i_var]
    file_name = f'Michel_TrackShower_{track_shower_vars[i_var].tree_name}'
    fig.savefig(f'{plot_dir}{file_name}.pdf', bbox_inches='tight')

# Save efficiency plots
for i_var in range(len(efficiency_vars)) :
    fig, _ = efficiency_plots[i_var]
    file_name = f'Michel_Efficiency_{efficiency_vars[i_var].tree_name}'
    fig.savefig(f'{plot_dir}{file_name}.pdf', bbox_inches='tight')